<a href="https://colab.research.google.com/github/JoseMunoz9/Portfolio/blob/main/FireOfficers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Artificial Intelligence Task

Part 1

In honour of the hard work that the fire officers are doing, the community has decided to build a new fire station for them. There are seven fire officers and four offices. Because there are more fire officers than offices, some fire officers have to be in the same offices as others. However, the fire officers are very picky about who they share with. The community administration is having trouble assigning fire officers to offices. They have asked you to plan where each fire officer goes.

The fire officers are Phylis, Ann, Henry, Eva, Bill, Mark, and Bob. They have given you the plans of the fire station: Each numbered area is an office in the fire station. Multiple officers can go into the same office, and not all offices have to be filled. Each fire person has restrictions about where they can be placed:

1. Phylis and Eva don’t get on, and do not want to be in the same office.
2. Mark and Bob are best friends, and have to be in the same office.
3. Henry listens to loud music. Only Eva will share his office.
4. Eva doesn't talk to Mark, Bob, and Bill.
5. Ann is always late but is a great fire officer. To hide that Ann is always late, Ann cannot be in either the same office or in an office adjacent to Phylis or Eva.
6. Phylis annoys Bill, so Bill doesn't want to be in Phyllis’s office.
7. Phylis is the fire chief, so she wants to be in office 1.

Using any CSP (Constraint Satisfaction Problem) framework, discover if the above problem can be solved and if so, detail who would be in each office.

In [1]:
# Source code reference: https://github.com/davecom/ClassicComputerScienceProblemsInPython/blob/master/Chapter3/csp.py
# csp.py
# From Classic Computer Science Problems in Python Chapter 3
# Copyright 2018 David Kopec
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from typing import Generic, TypeVar, Dict, List, Optional
from abc import ABC, abstractmethod

V = TypeVar('V') # variable type
D = TypeVar('D') # domain type


# Base class for all constraints
class Constraint(Generic[V, D], ABC):
    # The variables that the constraint is between
    def __init__(self, variables: List[V]) -> None:
        self.variables = variables

    # Must be overridden by subclasses
    @abstractmethod
    def satisfied(self, assignment: Dict[V, D]) -> bool:
        ...


# A constraint satisfaction problem consists of variables of type V
# that have ranges of values known as domains of type D and constraints
# that determine whether a particular variable's domain selection is valid
class CSP(Generic[V, D]):
    def __init__(self, variables: List[V], domains: Dict[V, List[D]]) -> None:
        self.variables: List[V] = variables # variables to be constrained
        self.domains: Dict[V, List[D]] = domains # domain of each variable
        self.constraints: Dict[V, List[Constraint[V, D]]] = {}
        for variable in self.variables:
            self.constraints[variable] = []
            if variable not in self.domains:
                raise LookupError("Every variable should have a domain assigned to it.")

    def add_constraint(self, constraint: Constraint[V, D]) -> None:
        for variable in constraint.variables:
            if variable not in self.variables:
                raise LookupError("Variable in constraint not in CSP")
            else:
                self.constraints[variable].append(constraint)

    # Check if the value assignment is consistent by checking all constraints
    # for the given variable against it
    def consistent(self, variable: V, assignment: Dict[V, D]) -> bool:
        for constraint in self.constraints[variable]:
            if not constraint.satisfied(assignment):
                return False
        return True

    def backtracking_search(self, assignment: Dict[V, D] = {}) -> Optional[Dict[V, D]]:
        # assignment is complete if every variable is assigned (our base case)
        if len(assignment) == len(self.variables):
            return assignment

        # get all variables in the CSP but not in the assignment
        unassigned: List[V] = [v for v in self.variables if v not in assignment]

        # get the every possible domain value of the first unassigned variable
        first: V = unassigned[0]
        for value in self.domains[first]:
            local_assignment = assignment.copy()
            local_assignment[first] = value
            # if we're still consistent, we recurse (continue)
            if self.consistent(first, local_assignment):
                result: Optional[Dict[V, D]] = self.backtracking_search(local_assignment)
                # if we didn't find the result, we will end up backtracking
                if result is not None:
                    return result
        return None

In [2]:
from typing import Dict, List, Optional

class FireOfficers (Constraint[str, str]):
    def __init__(self, place1: str, place2: str) -> None:
        super().__init__([place1, place2])
        self.place1: str = place1
        self.place2: str = place2

    def satisfied(self, assignment: Dict[str, str]) -> bool:
        if self.place1 not in assignment or self.place2 not in assignment:
            return True
        return assignment[self.place1] != assignment[self.place2]

In [3]:
if __name__ == "__main__":
    variables: List[str] = ["Phylis", "Eva", "Mark", "Bob", "Henry", "Bill", "Ann" ]
    domains: Dict[str, List[str]] = {}
    for variable in variables:
        domains[variable] = ["Room1", "Room2", "Room3", "Room4"]
    csp: CSP[str, str] = CSP(variables, domains)
    csp.add_constraint(FireOfficers("Eva","Mark")) # Eva doesn't talk to Mark, Bob, Bill and doesn't get on with Phylis.
    csp.add_constraint(FireOfficers("Eva", "Bob"))
    csp.add_constraint(FireOfficers("Eva", "Bill"))
    
    csp.add_constraint(FireOfficers("Ann", "Phylis")) # Ann cannot be in an office adjacent to Phylis or Eva.
    csp.add_constraint(FireOfficers("Ann", "Eva"))
    csp.add_constraint(FireOfficers("Ann", "Bob"))
    
    csp.add_constraint(FireOfficers("Phylis", "Eva")) # Phylis doesn't get on with Eva and she also annoys Bill.
    csp.add_constraint(FireOfficers("Phylis", "Bill"))
    
    csp.add_constraint(FireOfficers("Henry", "Phylis")) # Henry listens to loud music. Eva is the only one who would share his office.
    csp.add_constraint(FireOfficers("Henry", "Mark"))
    csp.add_constraint(FireOfficers("Henry", "Bob"))
    csp.add_constraint(FireOfficers("Henry", "Bill"))
    csp.add_constraint(FireOfficers("Henry", "Ann"))
    
    csp.add_constraint(FireOfficers("Mark", "Phylis")) # Mark is best friend with Bob, so they have to be in the same office.
    csp.add_constraint(FireOfficers("Mark", "Eva"))
    csp.add_constraint(FireOfficers("Mark", "Henry"))
    csp.add_constraint(FireOfficers("Mark", "Bill"))
    csp.add_constraint(FireOfficers("Mark", "Ann"))
    
    csp.add_constraint(FireOfficers("Bob", "Phylis")) # Bob is best friend with Mark, so they have to be in the same office.
    solution: Optional[Dict[str, str]] = csp.backtracking_search()
    if solution is None:
        print("No solution found!")
    else:
        print(solution)

{'Phylis': 'Room1', 'Eva': 'Room2', 'Mark': 'Room3', 'Bob': 'Room3', 'Henry': 'Room2', 'Bill': 'Room4', 'Ann': 'Room4'}


Solution:

Constraint Satisfaction Problem (CSP) has helped us to find the perfect solution for the problem above. See down below the details:


- 'Phylis' has been allocated to Office room 1 (fire chief).

- 'Eva and Henry' have been allocated to Office room 2 (Eva would only share office with Henry).

- 'Mark and Bob' have been allocated to Office room 3 (Mark and Bob are best friends).

- 'Ann and Bill' have been allocated to Office room 4 (Ann cannot be adjacent to Phylis nor Eva and Phylis annoys Bill).